<img src=https://static.wixstatic.com/media/f6bd9a_6a972d9b31324653bf198c38ce94339e~mv2.png>

# Carrusel de RDBMS
## Ejercicios de SQL resueltos

En este cuaderno resuelvo los ejercicios 41 al 50 del documento: 'Desarrollo de Aplicaciones en Entornos de Cuarta Generación y Herramientas CASE: Ejercicios de SQL.'

Para ello utilizo una version de la base de datos -que nombre "practicas_hospital"- modificada para ser compatible en esta ocasion, para **MS ACCESS**, siguiendo la estructura del libro.


Tanto el diagrama de **Entidad-Relacion**, como el **Script** para cargar la base de datos (que permite, se pueda replicar el ejercicio) en codigo VBA se encuentran disponibles en el mismo repositorio.

Como un bono extra a modo de ejercicio se importa todas las tablas de la base de datos a un archivo  y las consultas a **MS Excel**

#### Celda para configurar el entorno

In [ ]:
# !pip install pandas
# !pip install pyodbc
# !pip install openpyxl

#### Importamos librerias y se realiza la conexion utilizando los datos de la configuracion local de la base de datos

In [1]:
import pyodbc
import pandas as pd

# Crear la conexión
conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=C:\\Users\\ockis\\OneDrive\\Escritorio\\Nueva carpeta\\access\\practicas_hospital.accdb;')

# Crear un cursor
cursor = conn.cursor()


#### Realizamos una prueba y se Inspecciona la base de datos "practicas_hospital"

In [2]:
# Ejecutar la consulta e imprimir los resultados 
tablas = []
try:
    for table_info in cursor.tables(tableType='TABLE'):
        print(table_info.table_name)
        tablas.append(table_info.table_name)
    print("\nLa conexión se ha realizado con éxito.\nEstas son las tablas")
except Exception as e:
    print("\n Hubo un error al realizar la conexión:", e)

DEPARTAMENTO
DOCTOR
EMPLEADO
ENFERMO
HOSPITAL
OCUPACION
PLANTILLA
SALA

La conexión se ha realizado con éxito.
Estas son las tablas


### Resolucion de algunos ejercicios 

#####  41. Se desea obtener un listado combinación de los dos ejercicios anteriores (39 Y 40).

In [3]:
cont = 41

In [4]:
QUERY = """
SELECT E.EMP_NO AS EMPLEADO, E.APELLIDO AS NOMBRE, E.OFICIO AS OFICIO, E.DIR AS JEFE, E2.APELLIDO AS NOMBRE_J, E2.OFICIO AS OFICIO_J
FROM EMPLEADO AS E
LEFT JOIN EMPLEADO AS E2 ON E.DIR = E2.EMP_NO
UNION ALL
SELECT E.EMP_NO AS EMPLEADO, E.APELLIDO AS NOMBRE, E.OFICIO AS OFICIO, E.DIR AS JEFE, E2.APELLIDO AS NOMBRE_J, E2.OFICIO AS OFICIO_J
FROM EMPLEADO AS E
RIGHT JOIN EMPLEADO AS E2 ON E.DIR = E2.EMP_NO WHERE E.EMP_NO IS NULL
ORDER BY 2;
"""
cursor.execute(QUERY)
resultados = cursor.fetchall()
df = pd.DataFrame.from_records(resultados, columns=[column[0] for column in cursor.description])
df

,EMPLEADO,NOMBRE,OFICIO,JEFE,NOMBRE_J,OFICIO_J
0,NaN,None,None,NaN,Jimeno,Empleado
1,NaN,None,None,NaN,Alonso,Empleado
2,NaN,None,None,NaN,Tovar,Vendedor
3,NaN,None,None,NaN,Arenas,Vendedor
4,NaN,None,None,NaN,Sala,Vendedor
5,NaN,None,None,NaN,Arroyo,Vendedor
6,NaN,None,None,NaN,Sanchez,Empleado
7,NaN,None,None,NaN,Muñoz,Empleado
8,7876.0,Alonso,Empleado,7788.0,Gil,Analista
9,7654.0,Arenas,Vendedor,7698.0,Negro,Director


In [5]:
with pd.ExcelWriter('practicas_hospital.xlsx', mode='w') as writer:
    df.to_excel(writer, sheet_name=str(cont), index = False)
cont += 1

#### SUBCONSULTAS

##### 42. Obtener el apellido, departamento y oficio de aquellos empleados que tengan un oficio que este en el departamento 20 y que no sea ninguno de los oficios que esta en el departamento de VENTAS.

In [6]:
QUERY = """
SELECT APELLIDO, DEPT_NO, OFICIO
FROM EMPLEADO
WHERE DEPT_NO =20 AND OFICIO NOT IN (SELECT DISTINCT(E.OFICIO) AS OFICIO
                                    FROM EMPLEADO AS E
                                    INNER JOIN DEPARTAMENTO AS D 
                                    ON E.DEPT_NO = D.DEPT_NO
                                    WHERE D.DNOMBRE = 'VENTAS');
"""
cursor.execute(QUERY)
resultados = cursor.fetchall()
df = pd.DataFrame.from_records(resultados, columns=[column[0] for column in cursor.description])
df

,APELLIDO,DEPT_NO,OFICIO
0,Gil,20,Analista
1,Fernandez,20,Analista


In [7]:
with pd.ExcelWriter('practicas_hospital.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name=str(cont), index = False)
cont += 1

##### 43. Obtener el numero de empleado, numero de departamento y apellido de todos los empleados que trabajen en el departamento 20 o 30 y su salario sea mayor que dos veces el mínimo de la empresa. No queremos que el oficio sea PRESIDENTE.

In [8]:
QUERY = """
SELECT EMP_NO, DEPT_NO, APELLIDO
FROM EMPLEADO
WHERE DEPT_NO IN (20, 30) AND OFICIO <> 'PRESIDENTE' AND SALARIO > (SELECT 2*MIN(SALARIO)
                                                                    FROM EMPLEADO);
"""
cursor.execute(QUERY)
resultados = cursor.fetchall()
df = pd.DataFrame.from_records(resultados, columns=[column[0] for column in cursor.description])
df

,EMP_NO,DEPT_NO,APELLIDO
0,7566,20,Jimenez
1,7698,30,Negro
2,7788,20,Gil
3,7902,20,Fernandez


In [ ]:
with pd.ExcelWriter('practicas_hospital.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name=str(cont), index = False)
cont += 1

##### 44. Encontrar las personas que ganan 50000 PTA mas que el miembro del personal de sueldo mas alto del turno de mañana y que tenga el mismo trabajo que el Sr. Nuñez.

In [ ]:
QUERY = """
SELECT * 
FROM PLANTILLA
WHERE FUNCION = (SELECT FUNCION
FROM PLANTILLA
WHERE APELLIDO = 'NUÑEZ C.')
AND
SALARIO > (SELECT MAX(SALARIO) + 50000
FROM PLANTILLA
WHERE TURNO = 'M')
"""
cursor.execute(QUERY)
resultados = cursor.fetchall()
df = pd.DataFrame.from_records(resultados, columns=[column[0] for column in cursor.description])
df

In [ ]:
with pd.ExcelWriter('practicas_hospital.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name=str(cont), index = False)
cont += 1

##### 45. Queremos averiguar el apellido del individuo mas antiguo de la empresa.

In [ ]:
QUERY = """
SELECT APELLIDO 
FROM EMPLEADO
WHERE FECHA_ALTA = (SELECT MIN(FECHA_ALTA) FROM EMPLEADO);
"""
cursor.execute(QUERY)
resultados = cursor.fetchall()
df = pd.DataFrame.from_records(resultados, columns=[column[0] for column in cursor.description])
df

In [ ]:
with pd.ExcelWriter('practicas_hospital.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name=str(cont), index = False)
cont += 1

##### 46. Presentar los nombres y oficios de los empleados que tienen el mismo trabajo que JIMENEZ.

In [ ]:
QUERY = """
SELECT APELLIDO, OFICIO
FROM EMPLEADO
WHERE OFICIO =  (SELECT OFICIO FROM EMPLEADO WHERE UCASE(APELLIDO) = 'JIMENEZ' );
"""
cursor.execute(QUERY)
resultados = cursor.fetchall()
df = pd.DataFrame.from_records(resultados, columns=[column[0] for column in cursor.description])
df

In [ ]:
with pd.ExcelWriter('practicas_hospital.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name=str(cont), index = False)
cont += 1

##### 47. Queremos conocer el apellido, oficio, salario y departamento en el que trabajan, de todos los individuos cuyo salario sea mayor que el mayor salario del departamento 30.

In [ ]:
QUERY = """
SELECT APELLIDO, OFICIO, SALARIO
FROM EMPLEADO
WHERE SALARIO > (SELECT MAX(SALARIO) FROM EMPLEADO WHERE DEPT_NO = 30);
"""
cursor.execute(QUERY)
resultados = cursor.fetchall()
df = pd.DataFrame.from_records(resultados, columns=[column[0] for column in cursor.description])
df

In [ ]:
with pd.ExcelWriter('practicas_hospital.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name=str(cont), index = False)
cont += 1

##### 48. Presentar los nombres y oficios de todos los empleados del departamento 20, cuyo trabajo sea idéntico al de cualquiera de los empleados del departamento de VENTAS.

In [ ]:
QUERY = """
SELECT APELLIDO, OFICIO
FROM EMPLEADO
WHERE DEPT_NO = 20 AND OFICIO IN(SELECT DISTINCT(E.OFICIO)
                                FROM EMPLEADO AS E
                                INNER JOIN DEPARTAMENTO AS D
                                ON E.DEPT_NO = D.DEPT_NO
                                WHERE UCASE(DNOMBRE) = 'VENTAS');
"""
cursor.execute(QUERY)
resultados = cursor.fetchall()
df = pd.DataFrame.from_records(resultados, columns=[column[0] for column in cursor.description])
df

In [ ]:
with pd.ExcelWriter('practicas_hospital.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name=str(cont), index = False)
cont += 1

##### 49. Se desea obtener todos los empleados de los departamentos que no ganan ni el maximo ni el mínimo salarial de la empresa

In [ ]:
QUERY = """
SELECT * 
FROM EMPLEADO
WHERE SALARIO <> (SELECT MAX(SALARIO) FROM EMPLEADO) 
                AND  SALARIO <> (SELECT MIN(SALARIO) FROM EMPLEADO);
"""
cursor.execute(QUERY)
resultados = cursor.fetchall()
df = pd.DataFrame.from_records(resultados, columns=[column[0] for column in cursor.description])
df

In [ ]:
with pd.ExcelWriter('practicas_hospital.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name=str(cont), index = False)
cont += 1

##### 50. Se desea obtener el maximo salario por departamento, sin tener en cuenta a aquellos empleados cuyo apellido empieza con la inicial de alguno de los empleados que tienen el maximo salario de algún departamento. Tampoco queremos obtener los datos de departamentos con menos de 3 personas muestreadas

In [ ]:
QUERY = """
SELECT DEPT_NO, MAX(SALARIO) AS MAXIMO
FROM EMPLEADO
WHERE MID(APELLIDO,1,1) NOT IN (SELECT MID(APELLIDO,1,1) AS INICIAL FROM EMPLEADO WHERE SALARIO IN (SELECT  MAX(SALARIO)
                                                                                                    FROM EMPLEADO
                                                                                                    GROUP BY DEPT_NO))
GROUP BY DEPT_NO
HAVING COUNT(*) > 2;
"""
cursor.execute(QUERY)
resultados = cursor.fetchall()
df = pd.DataFrame.from_records(resultados, columns=[column[0] for column in cursor.description])
df

In [ ]:
with pd.ExcelWriter('practicas_hospital.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name=str(cont), index = False)
cont += 1

In [ ]:
for tabla in tablas:
    QUERY = f'SELECT * FROM {tabla};'
    cursor.execute(QUERY)
    resultados = cursor.fetchall()
    df = pd.DataFrame.from_records(resultados, columns=[column[0] for column in cursor.description])
    with pd.ExcelWriter('practicas_hospital.xlsx', mode='a') as writer:
        df.to_excel(writer, sheet_name=tabla, index = False)


In [ ]:
# Cierra la conexión
conn.close()

# Autor
### Oscar Gutierrez Leal
04 - 04 - 2024